# cyBERT: a flexible log parser based on the BERT language model

## Author
 - Rachel Allen, PhD (NVIDIA) [rachela@nvidia.com]
 
## Development Notes
* Developed using: RAPIDS v0.10.0 
* Last tested using: RAPIDS v0.10.0 on Nov 5, 2019

## Table of Contents
* Introduction
* Generating Labeled Logs
* Tokenization
* Data Loading
* Fine-tuning pretrained BERT
* Model Evaluation

## Introduction

One of the most arduous tasks of any security operation (and equally as time consuming for a data scientist) is ETL and parsing. This notebook illustrates how to train a BERT language model using previously parsed windows event logs as a labeled data set. We will fine-tune a pretrained BERT model with a classification layer for Named Entity Recognition.

In [13]:
import torch

from pytorch_transformers import BertTokenizer, BertModel, BertForTokenClassification, AdamW
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from seqeval.metrics import classification_report,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm,trange

import pandas as pd
import numpy as np

In [14]:
gdf = cudf.read_csv('/datasets/cyber/fake_win_events.csv')

In [15]:
gdf.eventcode.value_counts()

4624    1000
4625    1000
5156    1000
5157    1000
Name: eventcode, dtype: int32

In [20]:
mini4624 = gdf[gdf["eventcode"] == 4624].dropna(axis='columns')

In [21]:
mini4624.head()

,time,eventcode,uuid,computername,eventtype,keywords,logname,message,opcode,recordnumber,sourcename,taskcategory,type,insert_time,network_information_source_network_address,network_information_workstation_name,new_logon_account_domain,new_logon_account_name,subject_account_domain,subject_account_name
3000,1525108499000,4624,fc2f8767-9917-40ce-b1b2-19ac4ec80e76,laptop-70.wong.com,0,Audit Success,Security,An account was successfully logged on.,Info,1920306769,Microsoft Windows security auditing.,Logon,Information,10/04/2018 12:42:12 AM,198.49.63.55,tonyfreeman,stephens.biz,tonyfreeman,elliott.net,tonyfreeman
3001,1551582650000,4624,7d82817c-fc68-49ce-be41-8217a95e1f7c,lt-95.melton.com,0,Audit Success,Security,An account was successfully logged on.,Info,474033423,Microsoft Windows security auditing.,Logon,Information,02/28/2019 12:49:04 AM,192.175.54.118,gonzalespeter,blair.com,gonzalespeter,taylor.com,gonzalespeter
3002,1551625433000,4624,c660626e-b973-4d88-b0ff-3b1d183c9a71,email-28.norman.com,0,Audit Success,Security,An account was successfully logged on.,Info,116152764,Microsoft Windows security auditing.,Logon,Information,02/28/2019 01:27:14 AM,192.88.100.26,zacharydavis,mcbride-shelton.com,zacharydavis,clark.com,zacharydavis
3003,1551760436000,4624,81ada84b-feb3-4209-b669-d973de59e4a6,srv-58.lee.info,0,Audit Success,Security,An account was successfully logged on.,Info,134669246,Microsoft Windows security auditing.,Logon,Information,02/28/2019 01:27:14 AM,192.31.206.96,donna48,bell.info,donna48,campbell.com,donna48
3004,1548630137000,4624,2c0222f1-96f6-4051-988d-1e67f1c1ac71,srv-93.barron.info,0,Audit Success,Security,An account was successfully logged on.,Info,24976800615,Microsoft Windows security auditing.,Logon,Information,02/28/2019 12:49:05 AM,198.60.45.62,kathryn18,alexander-richardson.info,kathryn18,moore.biz,kathryn18


In [26]:
len(mini4624.dropna(axis='rows'))

1000

In [24]:
len(mini4624.dropna(axis='rows'))

1000

In [23]:
mini4624.columns

Index(['time', 'eventcode', 'uuid', 'computername', 'eventtype', 'keywords',
       'logname', 'message', 'opcode', 'recordnumber', 'sourcename',
       'taskcategory', 'type', 'insert_time',
       'network_information_source_network_address',
       'network_information_workstation_name', 'new_logon_account_domain',
       'new_logon_account_name', 'subject_account_domain',
       'subject_account_name'],
      dtype='object')

In [22]:
small_pdf = mini4624

In [ ]:
small_pdf['generated_raw'] = small_pdf['insert_time']+ " LogName= " + small_pdf['logname'].astype(str) \
                            + " SourceName= " + small_pdf['sourcename'].astype(str) \
                            + " EventCode= " + small_pdf['eventcode'].astype(str)  \
                            + " EventType= " + small_pdf['eventtype'].astype(str) \
                            + " Type= " + small_pdf['type'] \
                            + " ComputerName= " + small_pdf['computername'].astype(str) \
                            + " TaskCategory= " + small_pdf['taskcategory'].astype(str) \
                            + " OpCode= " + small_pdf['opcode'].astype(str)  \
                            + " RecordNumber= " + small_pdf['recordnumber'].astype(str) \
                            + " Keywords= " + small_pdf['keywords'].astype(str) \
                            + " Message= " + small_pdf['message'].astype(str) \

                            + "    Subject:   Account Name:  " + small_pdf['subject_account_name'].astype(str) \
                            + "   Account Domain:  " + small_pdf['subject_account_domain'].astype(str) \
                            
                            + "   New Logon:   Account Name:  " + small_pdf['new_logon_account_name'].astype(str) \
                            + "   Account Domain:  " + small_pdf['new_logon_account_domain'].astype(str) \
           
                            + " "
                            + "    Target Account:   Security ID:  " + small_pdf['target_account_security_id'].astype(str) \
                            + "   Account Name:  " + small_pdf['target_account_security_id'].str.split('\\').str[2] \
                            + "   Account Domain:  " + small_pdf['target_account_account_domain']

In [5]:
gdf.head()

,time,eventcode,uuid,computername,eventtype,keywords,logname,privileges,message,opcode,...,computer_account_that_was_changed_security_id,computer_account_that_was_changed_account_name,computer_account_that_was_changed_account_domain,changed_attributes_account_enabled,additional_information_caller_computer_name,account_information_account_domain,logon_information_logon_type,logon_information_elevated_token,logon_information_virtual_account,logon_information_restricted_admin_mode
0,1560935409000,5157,cb2d9118-4a13-436a-a437-9e3107030f81,email-20.chapman.biz,0,Audit Failure,Security,null,The Windows Filtering Platform has blocked a c...,Info,...,null,null,null,null,null,null,null,None,None,None
1,1560935410000,5157,5709cd57-2d3e-40d0-bba8-54bb8eb79a95,web-11.parks-olson.com,0,Audit Failure,Security,null,The Windows Filtering Platform has blocked a c...,Info,...,null,null,null,null,null,null,null,None,None,None
2,1560935504000,5157,340d250c-2f71-4487-aa54-60948fe7845c,web-90.smith.com,0,Audit Failure,Security,null,The Windows Filtering Platform has blocked a c...,Info,...,null,null,null,null,null,null,null,None,None,None
3,1560935510000,5157,9b20ce81-abb6-4cb2-8aa0-9cc111910645,email-28.kirk-hull.com,0,Audit Failure,Security,null,The Windows Filtering Platform has blocked a c...,Info,...,null,null,null,null,null,null,null,None,None,None
4,1560935519000,5157,a1fb3aed-b847-4603-b209-6e8786504638,email-62.cunningham.com,0,Audit Failure,Security,null,The Windows Filtering Platform has blocked a c...,Info,...,null,null,null,null,null,null,null,None,None,None


## Data prep
Initially we separate the logs by whitespace and create a label for every element from the split logs.

In [ ]:
##sample data
raw_logs = ["09/04/2019 05:58:30 PM LogName= Security SourceName= Microsoft Windows security auditing. EventCode= 4724 EventType= 0 Type= Information ComputerName= ZXJSDNFMAIL.acme.com \
            TaskCategory= User Account Management OpCode= Info RecordNumber= 16822951635 Keywords= Audit Success Message= An attempt was made to reset an account's password. Subject: \
            Security ID: NT AUTHORITY\\\\SYSTEM Account Name: DHDHMAIL107$ Account Domain: ACME.COM Logon ID: 0x3E7 Target Account: Security ID: DHDHMAIL107\\\\CLIUSER Account Name: \
            CLIUSR Account Domain: DHDHMAIL107",
            "09/04/2019 06:03:35 PM LogName= Security SourceName= Microsoft Windows security auditing. EventCode= 4725 EventType= 0 Type= Information ComputerName= SKFJMAIL.acme.com \
            TaskCategory= User Account Management OpCode= Info RecordNumber= 12434814940 Keywords= Audit Success Message= A user account was disabled. Subject: Security ID: NT \
            AUTHORITY\\\\SYSTEM Account Name: VGVMAIL104$ Account Domain: ACME.COM Logon ID: 0x3E7 Target Account: Security ID: VGVMAIL104\\\\CLIUSR Account Name: CLIUSR Account \
            Domain: VGVMAIL104"]
logs = [['09/04/2019','05:58:30','PM','LogName=','Security','SourceName=','Microsoft','Windows','security','auditing.','EventCode=','4724','EventType=','0','Type=','Information',
         'ComputerName=','ZXJSDNFMAIL.acme.com','TaskCategory=','User','Account','Management','OpCode=','Info','RecordNumber=','16822951635','Keywords=','Audit','Success',
         'Message=','An','attempt','was','made','to','reset','an',"account's",'password.','Subject:','Security','ID:','NT','AUTHORITY\\\\SYSTEM','Account','Name:','DHDHMAIL107$',
         'Account','Domain:','ACME.COM','Logon','ID:','0x3E7','Target','Account:','Security','ID:','DHDHMAIL107\\\\CLIUSER','Account','Name:','CLIUSR','Account','Domain:','DHDHMAIL107'],
        ['09/04/2019','06:03:35','PM','LogName=','Security','SourceName=','Microsoft','Windows','security','auditing.','EventCode=','4725','EventType=','0','Type=','Information',
         'ComputerName=','SKFJMAIL.acme.com','TaskCategory=','User','Account','Management','OpCode=','Info','RecordNumber=','12434814940','Keywords=','Audit','Success','Message=',
         'A','user','account','was', 'disabled.','Subject:','Security','ID:','NT','AUTHORITY\\\\SYSTEM','Account','Name:','VGVMAIL104$','Account','Domain:','ACME.COM','Logon',
         'ID:','0x3E7','Target','Account:','Security','ID:','VGVMAIL104\\\\CLIUSR','Account','Name:','CLIUSR','Account','Domain:','VGVMAIL104']
       ]
labels = [['time', 'time', 'time', 'key', 'logname', 'key', 'sourcename', 'sourcename', 'sourcename', 'sourcename', 'key', 'eventcode', 'key', 'eventtype', 'key', 'type', 'key', 
         'computername', 'key', 'taskcategory', 'taskcategory', 'taskcategory', 'key', 'opcode', 'key', 'recordnumber', 'key', 'keywords', 'keywords', 'key', 'message', 'message', 
         'message', 'message', 'message', 'message', 'message', 'message', 'message', 'key', 'key', 'key', 'subject_security_id', 'subject_security_id', 'key', 'key', 'subject_account_name', 
         'key', 'key', 'subject_account_domain', 'key', 'key', 'subject_logon_id', 'key', 'key', 'key', 'key', 'target_account_security_id', 'key', 'key', 'subject_logon_id', 'key', 'key', 
         'target_account_account_domain'],
        ['time', 'time', 'time', 'key', 'logname', 'key', 'sourcename', 'sourcename', 'sourcename', 'sourcename', 'key', 'eventcode', 'key', 'eventtype', 'key', 'type', 'key', 'computername', 
         'key', 'taskcategory', 'taskcategory', 'taskcategory', 'key', 'opcode', 'key', 'recordnumber', 'key', 'keywords', 'keywords', 'key', 'message', 'message', 'message', 'message', 
         'message', 'key', 'key', 'key', 'subject_security_id', 'subject_security_id', 'key', 'key', 'subject_account_name', 'key', 'key', 'subject_account_domain', 'key', 'key', 'subject_logon_id',
         'key', 'key', 'key', 'key', 'target_account_security_id', 'key', 'key', 'subject_logon_id', 'key', 'key', 'target_account_account_domain']]


We create a set list of all labels(tags) from our dataset, add `X` for wordpiece tokens we will not have tags for and `[PAD]` for logs shorter than the length of the model's embedding.

In [51]:
# set of tags
tag_values = list(set(x for l in labels for x in l))

# add 'X' tag for wordpiece 
tag_values.append('X')
tag_values.append('[PAD]')

# Set a dict for mapping id to tag name
tag2idx = {t: i for i, t in enumerate(tag_values)}

## Wordpiece tokenization
We are using the `bert-base-uncased` tokenizer from the pretrained BERT library from [HuggingFace](https://github.com/huggingface). This tokenizer splits our whitespace separated words further into in dictionary sub-word pieces. The model eventually uses the label from the first piece of a word as it's tag, so we do not care about the model's ability to predict labels for the sub-word pieces. For training, the tag used for these pieces is `X`. To learn more see the [BERT paper](https://arxiv.org/abs/1810.04805)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = []
new_labels = []
for sentence, tags in zip(logs,labels):
    new_tags = []
    new_text = []
    for word, tag in zip(sentence,tags):
        sub_words = tokenizer.wordpiece_tokenizer.tokenize(word.lower())
        for count, sub_word in enumerate(sub_words):
            if count > 0:
                tag = 'X'
            new_tags.append(tag)
            new_text.append(sub_word)
    tokenized_texts.append(new_text)
    new_labels.append(new_tags)

# Model inputs
For training our models needs (1) wordpiece tokens as integers padded to the specific length of the model (2) corresponding tags as integers and (3) a binary attention mask that ignores padding. Here we have have used 256 for the model size for each log or log piece. 

In [41]:
# convert string tokens into ints
input_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts]

# pad with input_ids with zeros and labels with [PAD]
def pad(l, content, width):
    l.extend([content] * (width - len(l)))
    return l

input_ids = [pad(x, 0, 256) for x in input_ids]

new_labels = [pad(x, '[PAD]', 256) for x in new_labels]


# attention mask for model to ignore padding
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]

# convert labels/tags to ints
tags = [[tag2idx.get(l) for l in lab] for lab in new_labels]

# Training and testing datasets
We split the data into training and validation sets.

In [42]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks = train_test_split(input_ids, tags, attention_masks, random_state=1234, test_size=0.1)

Move the datasets to the GPU

In [9]:
device = torch.device("cuda")

In [43]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

We create dataloaders to make batches of data ready to feed into the model. Here we use a batch size of 32.

In [44]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=32)

# Fine-tuning pretrained BERT

In [23]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))
#model to gpu
model.cuda();

In [24]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    #fine tune all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [25]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [49]:
epochs = 1
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss, scores = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss, logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask, labels=b_labels)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tag_values[p_i] for p in predictions for p_i in p]
    valid_tags = [tag_values[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))





Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Train loss: 0.0006899977425048344






Epoch: 100%|██████████| 1/1 [00:22<00:00, 22.22s/it]

Validation loss: 0.0003159592451993376
Validation Accuracy: 0.6264914772727272
F1-Score: 0.981049807144055


#### model eval

In [47]:
model.eval();

In [54]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
        
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
        
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    # Only predict the groud truth, mask=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # ground truth 
        temp_1 = []
        # Prediction
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mask=0 is PAD, do not compare
            if m: # Exclude the X label
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : 
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break      
        y_true.append(temp_1)
        y_pred.append(temp_2)

print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
print(classification_report(y_true, y_pred,digits=4))


f1 socre: 1.000000
Accuracy score: 1.000000
                               precision    recall  f1-score   support

target_account_account_domain     1.0000    1.0000    1.0000       150
                          key     1.0000    1.0000    1.0000      2700
       subject_account_domain     1.0000    1.0000    1.0000       150
                 taskcategory     1.0000    1.0000    1.0000       150
          subject_security_id     1.0000    1.0000    1.0000       150
             subject_logon_id     1.0000    1.0000    1.0000       300
                      logname     1.0000    1.0000    1.0000       150
                 recordnumber     1.0000    1.0000    1.0000       150
                 computername     1.0000    1.0000    1.0000       150
                      message     1.0000    1.0000    1.0000       150
                     keywords     1.0000    1.0000    1.0000       150
         subject_account_name     1.0000    1.0000    1.0000       150
   target_account_security_id   